In [81]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import cv2
import os
import re
import pickle
import math
from pathlib import *
import multiprocessing
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from collections import *
from itertools import *
from functools import *
from sklearn.metrics import *
from scipy.stats import *
import pandas as pd
import seaborn as sns
import hashlib
from PIL import Image
from datastatics import *
from oct_Utils import *
import itertools
from torchvision.ops import RoIPool
import torchvision.models as models
from torchsummary import summary


ModuleNotFoundError: No module named 'utils'

In [1]:
def generate_anchor_base(base_size=16, ratios=[0.5, 1, 2],
                         anchor_scales=[8, 16, 32]):

    py = base_size / 2.
    px = base_size / 2.

    anchor_base = np.zeros((len(ratios) * len(anchor_scales), 4),
                           dtype=np.float32)
    for i in range(len(ratios)):
        for j in range(len(anchor_scales)):
            h = base_size * anchor_scales[j] * np.sqrt(ratios[i])
            w = base_size * anchor_scales[j] * np.sqrt(1. / ratios[i])


            index = i * len(anchor_scales) + j
            anchor_base[index, 0] = py - h / 2.
            anchor_base[index, 1] = px - w / 2.
            anchor_base[index, 2] = py + h / 2.
            anchor_base[index, 3] = px + w / 2.
    return anchor_base

In [4]:
anchor=generate_anchor_base()

In [120]:
def generate_vf_proposal():
    centre_point_x=np.linspace((224/54)/2,224-(224/54)/2,54)
    centre_point_y=np.repeat(112,54)
    w=224/54
    h=224
    temp=[(centre_point_y-h/2).reshape(54,-1),(centre_point_x-w/2).reshape(54,-1),(centre_point_y+h/2).reshape(54,-1),(centre_point_x+w/2).reshape(54,-1)]
    proposal=np.concatenate(temp,axis=1)
    return proposal
    #return np.concatenate([centre_point_y-,])

In [121]:
bupt=generate_vf_proposal()

In [122]:
bupt

array([[  0.        ,   0.        , 224.        ,   4.14814815],
       [  0.        ,   4.14814815, 224.        ,   8.2962963 ],
       [  0.        ,   8.2962963 , 224.        ,  12.44444444],
       [  0.        ,  12.44444444, 224.        ,  16.59259259],
       [  0.        ,  16.59259259, 224.        ,  20.74074074],
       [  0.        ,  20.74074074, 224.        ,  24.88888889],
       [  0.        ,  24.88888889, 224.        ,  29.03703704],
       [  0.        ,  29.03703704, 224.        ,  33.18518519],
       [  0.        ,  33.18518519, 224.        ,  37.33333333],
       [  0.        ,  37.33333333, 224.        ,  41.48148148],
       [  0.        ,  41.48148148, 224.        ,  45.62962963],
       [  0.        ,  45.62962963, 224.        ,  49.77777778],
       [  0.        ,  49.77777778, 224.        ,  53.92592593],
       [  0.        ,  53.92592593, 224.        ,  58.07407407],
       [  0.        ,  58.07407407, 224.        ,  62.22222222],
       [  0.        ,  62

In [106]:
from oct_vf_model import OCTVFModel
from torchsummary import summary
import torchvision.models as models

def generate_vf_proposal():
    centre_point_x=np.linspace((224/54)/2,224-(224/54)/2,54)
    centre_point_y=np.repeat(112,54)
    w=224/54
    h=224
    temp=[(centre_point_y-h/2).reshape(54,-1),(centre_point_x-w/2).reshape(54,-1),(centre_point_y+h/2).reshape(54,-1),(centre_point_x+w/2).reshape(54,-1)]
    proposal=np.concatenate(temp,axis=1)
    return proposal
    #return np.concatenate([centre_point_y-,])

def normal_init(m, mean, stddev, truncated=False):
    """
    weight initalizer: truncated normal and random normal.
    """
    # x is a parameter
    if truncated:
        m.weight.data.normal_().fmod_(2).mul_(stddev).add_(mean)  # not a perfect approximation
    else:
        m.weight.data.normal_(mean, stddev)
        m.bias.data.zero_()

def decom_ResNet50():
    model=models.ResNet50(pretrained=True)
    feature=nn.Sequential(**list(model.children()[:-3]))
    reg_layer=nn.Sequential(
        model.avgpool,
        nn.Linear(512,1000)#输出的feature是1000
    )
    return feature,reg_layer
    
class OCT_ROI_head(nn.Module):

    def __init__(self, classifier, roi_size=7, spatial_scale=16):
        super(OCT_ROI_head, self).__init__()
        self.feature,self.reg_layer=decom_ResNet50()
        self.vf_pred = nn.Linear(4096, 1)
        normal_init(self.vf_pred, 0, 0.01)
        self.roi_size = roi_size
        self.spatial_scale = spatial_scale
        self.roi = RoIPool( (self.roi_size, self.roi_size),self.spatial_scale)
 
    def forward(self, x, rois, roi_indices):
        roi_indices =torch.tensor(roi_indices).float()
        rois = torch.tensor(rois).float()
        indices_and_rois = torch.cat([roi_indices[:, None], rois], dim=1)
        xy_indices_and_rois = indices_and_rois[:, [0, 2, 1, 4, 3]]
        pool = self.roi(x, indices_and_rois)
        fc7 = self.classifier(pool)
        vf_predicted_value = self.vf_pred(fc7)
        
        return vf_predicted_value


In [97]:
extractor, classifier = decom_ResNet50()
net=OCT_ROI_head(roi_size=7,spatial_scale=(1./feat_stride),classfier=classifier)

In [116]:

model=models.resnet50(pretrained=True)


In [119]:
summary(model.cuda(),input_size=(3,224,224),batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 112, 112]           9,408
       BatchNorm2d-2          [1, 64, 112, 112]             128
              ReLU-3          [1, 64, 112, 112]               0
         MaxPool2d-4            [1, 64, 56, 56]               0
            Conv2d-5            [1, 64, 56, 56]           4,096
       BatchNorm2d-6            [1, 64, 56, 56]             128
              ReLU-7            [1, 64, 56, 56]               0
            Conv2d-8            [1, 64, 56, 56]          36,864
       BatchNorm2d-9            [1, 64, 56, 56]             128
             ReLU-10            [1, 64, 56, 56]               0
           Conv2d-11           [1, 256, 56, 56]          16,384
      BatchNorm2d-12           [1, 256, 56, 56]             512
           Conv2d-13           [1, 256, 56, 56]          16,384
      BatchNorm2d-14           [1, 256,

In [112]:
model.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [ ]:
toch.summary(model,)

In [90]:
def decom_vgg16():
    # the 30th layer of features is relu of conv5_3
    model=models.vgg16()

    features = list(model.features)[:30]
    classifier = model.classifier

    classifier = list(classifier)

    classifier = nn.Sequential(*classifier)

    # for layer in features[:10]:
    #     for p in layer.parameters():
    #         p.requires_grad = False

    return nn.Sequential(*features), classifier

In [91]:
a,b=decom_vgg16()

In [94]:
tem=a(torch.rand((3,600,1000)).unsqueeze(0))

In [101]:
b

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [96]:
b

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
summary(model,input_size=(3,600,1000),batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [1, 64, 600, 1000]           1,792
              ReLU-2         [1, 64, 600, 1000]               0
            Conv2d-3         [1, 64, 600, 1000]          36,928
              ReLU-4         [1, 64, 600, 1000]               0
         MaxPool2d-5          [1, 64, 300, 500]               0
            Conv2d-6         [1, 128, 300, 500]          73,856
              ReLU-7         [1, 128, 300, 500]               0
            Conv2d-8         [1, 128, 300, 500]         147,584
              ReLU-9         [1, 128, 300, 500]               0
        MaxPool2d-10         [1, 128, 150, 250]               0
           Conv2d-11         [1, 256, 150, 250]         295,168
             ReLU-12         [1, 256, 150, 250]               0
           Conv2d-13         [1, 256, 150, 250]         590,080
             ReLU-14         [1, 256, 1